In [ ]:

## versions:
## Python    : 3.11.5
## numpy     : 1.26.0
## torch     : 2.1.0
## pandas    : 2.1.1

# licensed under the Creative Commons - Attribution-NonCommercial 4.0
# International license (CC BY-NC 4.0):
# https://creativecommons.org/licenses/by-nc/4.0/. 

import os
import io
import sys
import shutil
import datetime
from typing import Dict, List, Optional
from copy import deepcopy

import numpy as np
import pandas as pd
import torch as t
from torch.utils.data import DataLoader
from scipy import stats
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

from common.torch.ops import empty_gpu_cache
from common.sampler import ts_dataset
from common.torch.snapshots import SnapshotManager
from experiments.trainer import trainer_var
from experiments.model import generic_dec_var
from models.exog import TCN_encoder

from data_utils.forecast import tryJSON, Struct, read_config, default_settings, make_training_fn
from data_utils.forecast import init_target_data, load_exog_data, make_training_fn, generate_quantiles
from data_utils.forecast import pickle_results, read_pickle, output_figs
from data_utils.flu import domain_defaults, specify_ensemble, custom_ensemble, output_df, append_forecasts
from data_utils.flu import read_flu_data, read_weather_data


In [ ]:
import warnings
%config InlineBackend.figure_formats = ["svg"]
plt.style.use("dark_background")
warnings.formatwarning = lambda message, category, *args, **kwargs: "{}: {}\n".format(category.__name__, message)
warnings.filterwarnings("ignore",category=FutureWarning)
#%load_ext watermark
#%watermark -n -u -v -iv -w

(if needed) read latest data

In [ ]:
#idx, _ = read_flu_data()
#read_weather_data(idx)

`read_config()` returns configuration settings that don't change between models within an ensemble

gets values from `config.json` if available

see comments in `data_utils/forecast.py` for an explanation of entries

In [ ]:
rstate = read_config("config_flu.json")

In [ ]:
rstate

you can change the settings here or in `config.json`

e.g., `rstate.cut` sets the train/test split index (None = train on all data)

In [ ]:
rstate.cut = 447 #395 #432 #436 #None  #

`default_settings()` returns settings that can be changed between models within an ensemble

gets defaults from `settings.json` if available

see comments in `data_utils/forecast.py` for an explanation of entries

In [ ]:
settings = default_settings("settings_flu.json")

can change settings in json file or here

In [ ]:
#try increasing the learning rate when there's more training data
#settings.init_LR = np.round(0.0001 + (rstate.cut - 901) * 4e-7, 7) 

we will change `settings.exog_vars` below, to specify which exogenous predictors to use

In [ ]:
settings

`domain_defaults()` is meant to be a user-defined function

returns a struct with instructions for reading or generating exogenous variables

see `data_utils/covid_hub.py` for an example/explanation

In [ ]:
domain_specs = domain_defaults()

`exog_vars` specifies which exogenous predictors to use by default

the predictors in `var_names` are loaded/generated and available to use

In [ ]:
domain_specs

`init_target_data()` reads in and optionally transforms target data

sets timepoint indices and series identifiers; writes data to `rstate`


In [ ]:
rstate, settings = init_target_data(rstate, settings)

`rstate.data_index` was set based on the index of `rstate.target_file`

for exogenous data, the files and functions specified in `domain_defaults()` must generate data frames with the same index

In [ ]:
rstate.data_dir+"/"+rstate.target_file, rstate.data_index

`load_exog_data()` appends exogenous predictors to rstate, using the data index generated above

In [ ]:
rstate, settings = load_exog_data(rstate, settings, domain_specs)

`settings.exog_vars` now has the defaults from domain_specs (if this was not set in `settings.json`)

In [ ]:
settings.exog_vars

the data has been read into `rstate` as a dict keyed by series name

each series is a data frame with rows as timepoints and columns as variables

In [ ]:
rstate.series_dfs["Maryland"]

the name of the target column was set automatically by `init_target_data()`

In [ ]:
rstate.target_var

`make_training_fn()` returns a function that trains a model  (it closes over training data and config settings)

the resulting function takes `settings` and returns mean & variance forecasts

the forecasts are matrices with rows = series and columns = timepoints

the trained models are saved in `rstate.snapshot_dir`

the training function can be used on its own or called in a loop with different settings to generate an ensemble


In [ ]:
training_fn = make_training_fn(rstate)

set settings.cut_weights to increase training effort on rare events such as prominent peaks

In [ ]:

def peak_cut_weights(rstate, settings):

    ## use scipy find_peaks to find indices of qualifying peaks
    idx_pre_peak = []; idx_immed_peak = []; idx_post_peak = []; idx_zeros = []; idx_highest_peak = []; idx_post_highest = [] #;series_peaks = []
    for targ_sum in rstate.nat_targets:
        peaks,_ = find_peaks(targ_sum,height=2.0,distance=settings.horizon)
        #peaks,_ = find_peaks(targ_sum,height=2.0,prominence=0.75)
        #series_peaks.append(peaks)
        ## highest peak of each season
        highest_peaks,_ = find_peaks(targ_sum,height=2.0,distance=25)

        ## cut points that place a peak inside the forecast horizon
        idx_pre_peak.append( [i for i in np.unique(np.concatenate([range(x - settings.horizon + 1, x) for x in peaks])) if (0 <= i < targ_sum.shape[0])] )
        
        ## n cutpoints before a peak
        idx_immed_peak.append( [i for i in np.unique(np.concatenate([range(x - 3, x) for x in peaks])) if (0 <= i < targ_sum.shape[0])] )

        ## n cutpoints before highest peak
        idx_highest_peak.append( [i for i in np.unique(np.concatenate([range(x - 3, x) for x in highest_peaks])) if (0 <= i < targ_sum.shape[0])] )

        ## cut points at a peak and a few points after
        idx_post_peak.append( [i for i in np.unique(np.concatenate([range(x, x + 4) for x in peaks])) if (0 <= i < targ_sum.shape[0])] )

        ## n cutpoints at and after highest peak
        idx_post_highest.append( [i for i in np.unique(np.concatenate([range(x, x + 6) for x in highest_peaks])) if (0 <= i < targ_sum.shape[0])] )

        ## points where the whole horizon is near zero
        #idx_zeros.append( [i for i in range(targ_sum.shape[0]) if np.all(targ_sum[i:i+settings.horizon+1] < 0.25)] )
        ## just points near zero
        idx_zeros.append( [i for i in range(targ_sum.shape[0]) if targ_sum[i]<0.25] )

    cut_weights = [None for _ in rstate.nat_targets]

    for (i,targ_sum) in enumerate(rstate.nat_targets):
        W = np.ones(targ_sum.shape[0])
        ## careful of overwrite order:
        #W[idx_post_peak[i]] = 1.0 #2.0 #4.0 #2.0
        #W[idx_pre_peak[i]] = 2.0 #2.0
        #W[idx_immed_peak[i]] = 1.0 #2.0

        ## "peakfinder" model
        W[idx_post_highest[i]] = 4.0 
        W[idx_highest_peak[i]] = 8.0 

        W[idx_zeros[i]] = 0.25 #0.1 # 
        cut_weights[i] = W

    return cut_weights
 


In [ ]:
#settings.cut_weights = None

settings.cut_weights = peak_cut_weights(rstate, settings)

In [ ]:
i = 5
targ_sum = rstate.nat_targets[i]
_,ax=plt.subplots()
plt.plot(targ_sum,linewidth=0.75);
wts = settings.cut_weights[i];
i0 = np.nonzero(wts<0.99)[0]
i1 = np.nonzero(wts>1.01)[0]
i2 = np.nonzero(wts>2.01)[0]
i3 = np.nonzero(wts>4.01)[0]
plt.plot(i0,targ_sum[i0],".",alpha=0.5,color="orangered");
plt.plot(i1,targ_sum[i1],".",color="white");
plt.plot(i2,targ_sum[i2],".",color="orange");
plt.plot(i3,targ_sum[i3],"+",color="magenta");
#plt.plot(series_peaks[i], targ_sum[series_peaks[i]],"x",alpha=0.66); #plt.plot(idx_zeros[i],targ_sum[idx_zeros[i]],".",alpha=0.5,color="orangered");
#plt.plot(idx_post_peak[i],targ_sum[idx_post_peak[i]],".",color="orange"); #plt.plot(idx_pre_peak[i],targ_sum[idx_pre_peak[i]],"+",color="white");
#plt.plot(idx_immed_peak[i],targ_sum[idx_immed_peak[i]],".",color="white"); #ax.set_xlim([120,220])


to use snapshot/pretrained model with no additional training, set iterations to 0

In [ ]:
## use snapshot/pretrained model, no additional training
#settings.iterations = 0


to train an ensemble of models, we will generate a list of `settings`, one for each model

`specify_ensemble` is a user-defined function that generates the list, based on info in `domain_specs`

see `data_utils/covid_hub.py` for an example

In [ ]:
domain_specs.random_reps = 2

## generate a list of settings structs having the desired variation for ensemble
## save the list to rstate for posterity
rstate.settings_list = specify_ensemble(settings, domain_specs)


can also define some other ensemble:

In [ ]:
## setting size of hidden layer based on size of lookback window:
## (defined in flu.py)

rstate.settings_list = custom_ensemble(settings, domain_specs)

In [ ]:
rstate.settings_list[3]

(optional) a pretrained model file for each model in the ensemble

each must have the same structure (lookback window, hidden dims, etc.) as the corresponding ensemble entry

In [ ]:

def pretrained_list(pretrain_dir, specs):
    file_list = []
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filename = "flu24_" + str(opt) + "H_" + str(j+1) + ".pt"
            file_list.append(os.path.join(pretrain_dir,filename))
    return file_list


In [ ]:
rstate.pretrained_models = [None for x in rstate.settings_list]

pretrain_dir = None # "flu_pretrained_2024" # 

if pretrain_dir is not None:
    rstate.pretrained_models = pretrained_list(os.path.join(rstate.data_dir,pretrain_dir), domain_specs)

rstate.pretrained_models

empty dicts for storing the forecasts from each model:

In [ ]:
mu_fc={}
var_fc={}

In [ ]:
empty_gpu_cache() ## just in case?

train each model in the ensemble and write its forecast to `mu_fc` and `var_fc` (keyed w/ a semi-descriptive name):

In [ ]:

## ensemble loop
for i, set_i in enumerate(rstate.settings_list):
    model_name = rstate.output_prefix+"_"+str(i)
    model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
    model_name = model_name+"_"+model_suffix
    print("training ",model_name)
    mu_fc[model_name], var_fc[model_name] = training_fn(model_name, set_i, rstate.pretrained_models[i]) 


forecast shape for each model is [series, time]

ensemble the dict values using median across models

write results to `rstate`

In [ ]:

mu_fc["ensemble"] = np.median(np.stack([mu_fc[k] for k in mu_fc]),axis=0)
var_fc["ensemble"] = np.median(np.stack([var_fc[k] for k in var_fc]),axis=0)

rstate.mu_fc = mu_fc
rstate.var_fc = var_fc


if forecast targets are per-capita, need series weights for summing to national (per capita) forecast

In [ ]:
if rstate.series_weights is not None:
    print(pd.DataFrame({"state":rstate.series_names,"weight":rstate.series_weights.squeeze()}))

`generate_quantiles()` goes through each entry in `rstate.mu_fc` and `rstate.var_fc`

and generates dicts containing forecast quantiles for each model (and "ensemble")

see comments in `data_utils/forecast.py` for details

In [ ]:
rstate = generate_quantiles(rstate)

optional: save rstate, which contains all training data, forecasts, and ensemble settings

`pickle_results()` writes it to output dir

In [ ]:
pickle_results(rstate)

plot some forecasts

In [ ]:
output_figs(rstate, rstate.settings_list[0].horizon, 
#[0,1,2,4], 
#[8,9,10,11], 
range(12),
 70,
 colors=["white","yellow"],figsize=(5,3),plot_mean=True)

save forecasts as csv

In [ ]:
df,_ = output_df(rstate,0)
df.query("location == 'US' and quantile=='mean'")


In [ ]:
#append_forecasts(df, os.path.join(rstate.data_dir,"forecast_plots.csv"))


delete the trained models if we no longer need them:

In [ ]:
if rstate.delete_models:
    try:
        shutil.rmtree(rstate.snapshot_dir)
    except:
        pass


automate the above

In [ ]:
def init_rstate(cut, settings, domain_specs, ensemble_fn=specify_ensemble, cut_weight_fn=None):
    rstate = read_config("config_flu.json")
    rstate.cut = cut
    
    rstate, settings = init_target_data(rstate, settings)
    rstate, settings = load_exog_data(rstate, settings, domain_specs)

    if cut_weight_fn is not None: settings.cut_weights = cut_weight_fn(rstate, settings)

    rstate.settings_list = ensemble_fn(settings, domain_specs)
    rstate.pretrained_models = [None for x in rstate.settings_list]
    
    return rstate, settings


def generate_ensemble(rstate, ens_fn=np.median):
    mu_fc={}
    var_fc={}
    empty_gpu_cache()
    training_fn = make_training_fn(rstate)

    ## ensemble loop
    for i, set_i in enumerate(rstate.settings_list):
        model_name = rstate.output_prefix+"_"+str(i)
        model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
        model_name = model_name+"_"+model_suffix
        print("training ",model_name)
        mu_fc[model_name], var_fc[model_name] = training_fn(model_name, set_i, rstate.pretrained_models[i]) 

    mu_fc["ensemble"] = ens_fn(np.stack([mu_fc[k] for k in mu_fc]),axis=0)
    var_fc["ensemble"] = ens_fn(np.stack([var_fc[k] for k in var_fc]),axis=0)
    rstate.mu_fc = mu_fc
    rstate.var_fc = var_fc

    rstate = generate_quantiles(rstate)

    return rstate


def delete_model_dir(rstate):
    if rstate.delete_models:
        try:
            shutil.rmtree(rstate.snapshot_dir)
        except:
            pass


In [ ]:

## try adjusting the amount of training based on the amount of training data history
## (lowering learning rate seems to work better than decreasing # of iterations)
def adapt_iter(x):
    return None#int(np.round(200 + (x - 901) * 2.0 / 3.0))

def adapt_lr(x):
    return None#np.round(0.0001 + (x - 901) * 4e-7, 7) 

def run_test(cut, random_reps=None, ensemble_fn=specify_ensemble, series_figs=[], n_iter=None, pretrain_list_fn=None, cut_weight_fn=None, ens_reduce=np.median, adj_iter=False, adj_LR=False):
    ## if adj_*, train more when there is more data; otherwise use values from settings.json
    settings = default_settings("settings_flu.json")
    #if adj_iter: settings.iterations = adapt_iter(cut)
    #if adj_LR: settings.init_LR = adapt_lr(cut)
    if n_iter is not None: settings.iterations = n_iter

    domain_specs = domain_defaults()
    if random_reps is not None: domain_specs.random_reps = random_reps
    
    rstate, settings = init_rstate(cut, settings, domain_specs, ensemble_fn, cut_weight_fn)

    if pretrain_list_fn is not None:
        rstate.pretrained_models = pretrain_list_fn(rstate.data_dir, domain_specs)

    rstate = generate_ensemble(rstate, ens_reduce)

    pickle_results(rstate)
    output_figs(rstate, rstate.settings_list[0].horizon, 
                series_figs, 
                70,
                colors=["white","yellow"],figsize=(5,3),plot_mean=True)

    df, date_stamp = output_df(rstate,0)

    delete_model_dir(rstate)
    
    return (df, date_stamp)


graph training losses

note, ensembling not-quite-converged models seems to work better than running more iterations


In [ ]:

def plot_losses(pickle_file,ylim=None):
    rstate = read_pickle(pickle_file)
    model_prefix = rstate.output_prefix
    model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
    _, ax = plt.subplots(nrows=len(rstate.settings_list),ncols=2,figsize=[8,2*len(rstate.settings_list)])
    for i, set_i in enumerate(rstate.settings_list):
        model_name =  model_prefix+"_"+str(i)+"_"+model_suffix
        total_iter = set_i.iterations
        snapshot_manager = SnapshotManager(snapshot_dir=os.path.join(rstate.snapshot_dir, model_name), total_iterations=total_iter)
        ldf = snapshot_manager.load_training_losses()
        vdf = snapshot_manager.load_validation_losses()
        ax[i,0].plot(ldf)
        ax[i,1].plot(vdf)
        ax[i,1].set_ylim(ylim)
    #plt.show()
    plt.savefig(os.path.join(rstate.output_dir , "losses_"+model_prefix+"_"+model_suffix+".png"))


In [ ]:
plot_losses(os.path.join("fluview", "output", "flu_395.pickle"))

In [ ]:
plot_losses(os.path.join("fluview", "output", "flu_447.pickle"))

In [ ]:
#run_test(cut=436, random_reps=3, ensemble_fn=custom_ensemble, series_figs=range(12), cut_weight_fn=peak_cut_weights)

In [ ]:
#plot_losses(os.path.join("fluview", "output", "flu_436.pickle"))

In [ ]:
def pre23test(data_dir, specs):
    file_list = []
    i = 0
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filepath = os.path.join("flu_"+str(i)+"_395","model")
            file_list.append(os.path.join(data_dir,"snapshots",filepath))
            i = i + 1
    return file_list

def pre24test(data_dir, specs):
    file_list = []
    i = 0
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filepath = os.path.join("flu_"+str(i)+"_447","model")
            file_list.append(os.path.join(data_dir,"snapshots",filepath))
            i = i + 1
    return file_list

def pre24pf(data_dir, specs):
    file_list = []
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filepath = "flu24_"+str(opt)+"H_"+str(j+1)+".pt"
            file_list.append(os.path.join(data_dir,"peakfinder_pretrained",filepath))
    return file_list

def pre24ond(data_dir, specs):
    file_list = []
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filepath = "flu24_"+str(opt)+"H_"+str(j+1)+".pt"
            file_list.append(os.path.join(data_dir,"orig_noise_downwt_pretrained",filepath))
    return file_list


In [ ]:
csv_name = "forecast_peakfinder"
df,_ = run_test(None, 3, custom_ensemble, [], 0, pre24pf)
append_forecasts(df, os.path.join("fluview",csv_name+".csv"))

In [ ]:
csv_name = "forecast_orig_noise_down"
df,_ = run_test(None, 3, custom_ensemble, [], 0, pre24ond)
append_forecasts(df, os.path.join("fluview",csv_name+".csv"))

In [ ]:
csv_name = "test"
last_idx = 473

In [ ]:
for cut in range(415,448):
    df,_ = run_test(cut, 3, custom_ensemble, [], 0, pre23test);
    append_forecasts(df, os.path.join("fluview",csv_name+".csv"))

In [ ]:
shutil.copyfile(os.path.join("fluview",csv_name+".csv"), os.path.join("fluview",csv_name+"_2023.csv"))

for cut in range(467,last_idx):
    df,_ = run_test(cut, 3, custom_ensemble, [], 0, pre24test)
    append_forecasts(df, os.path.join("fluview",csv_name+".csv"))
    
df,_ = run_test(None, 3, custom_ensemble, [], 0, pre24test)
append_forecasts(df, os.path.join("fluview",csv_name+".csv"))

In [ ]:
#rstate.delete_models = True
#delete_model_dir(rstate)

In [ ]:
## pull pretrained models out of snapshot directories

model_dir = "snapshots"
n = 2
opts = [2,3,4,5]
idx = 4

i = 0
for j in range(n):
    for opt in opts:
        filepath = os.path.join("fluview",model_dir,"flu_"+str(i)+"_447","model")
        dest = os.path.join("fluview",model_dir,"flu24_"+str(opt)+"H_"+str(j+idx)+".pt")
        shutil.copyfile(filepath, dest)
        i = i + 1
i = 0
for j in range(n):
    for opt in opts:
        filepath = os.path.join("fluview",model_dir,"flu_"+str(i)+"_395","model")
        dest = os.path.join("fluview",model_dir,"flu23_"+str(opt)+"H_"+str(j+idx)+".pt")
        shutil.copyfile(filepath, dest)
        i = i + 1
